
# Bioluminescence SQL Analysis

This notebook explores data related to bioluminescent organisms. It performs several SQL queries to extract meaningful insights, ranging from simple counts to complex categorization and aggregation.

---


## 1. Count of Sightings per Taxon

Counting how many organisms have bioluminecent sightings is a quick way to find some of the most common bioluminecnet organisms and therefor is useful when doing research on what to look out for 

In [ ]:

SELECT taxon_name, COUNT(*) AS sighting_count
FROM bioluminescent_organisms
GROUP BY taxon_name
ORDER BY sighting_count DESC;


## 2. Most Common Sighting per Country

Finding the most common sighting of bioluminecent organism is not as useful unless it is localized to the users location.  Finding the most spotted organism by country is the answer to this.

In [ ]:

WITH ranked_taxa AS (
    SELECT country, taxon_name, COUNT(*) AS sightings,
           RANK() OVER (PARTITION BY country ORDER BY COUNT(*) DESC) AS rnk
    FROM bioluminescent_organisms
    GROUP BY country, taxon_name
)
SELECT country, taxon_name, sightings
FROM ranked_taxa
WHERE rnk = 1;


## 3. Max Sightings per Taxon per Month

Another very important distinction is finding the most common month for sightings of bioluminecent organisms considering that they only apear at very specific times of the year.  This helps identify when you should go out to find your organism.  

In [ ]:

WITH sightings_per_month AS (
    SELECT taxon_name, strftime('%Y-%m', sighting_date) AS month, COUNT(*) AS sightings
    FROM bioluminescent_organisms
    GROUP BY taxon_name, month
),
ranked_months AS (
    SELECT *, RANK() OVER (PARTITION BY taxon_name ORDER BY sightings DESC) AS rnk
    FROM sightings_per_month
)
SELECT taxon_name, month, sightings
FROM ranked_months
WHERE rnk = 1;


## 4. Categorizing Organisms by Light Level

Here we merge a table of light level for each organism to find which ones are bright and therefor easier to spot when searching. 

In [ ]:

WITH merged AS (
    SELECT b.*, l.lumens
    FROM bioluminescent_organisms b
    JOIN lumens_table l ON b.taxon_name = l.taxon_id
),
stats AS (
    SELECT 
        PERCENTILE_CONT(0.33) WITHIN GROUP (ORDER BY lumens) AS pct33,
        PERCENTILE_CONT(0.66) WITHIN GROUP (ORDER BY lumens) AS pct66
    FROM merged
)
SELECT m.taxon_name, m.lumens,
       CASE 
           WHEN m.lumens <= s.pct33 THEN 'weak'
           WHEN m.lumens <= s.pct66 THEN 'moderate'
           ELSE 'strong'
       END AS light_category
FROM merged m, stats s;
